In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# loading dataset
df = pd.read_csv("/train.csv")

# **Question A**

In [ ]:
# removing units and convert to float

df['Mileage'] = (
    df['Mileage']
    .str.extract(r'([\d\.]+)')
    .astype(float)
)


df['Engine'] = (
    df['Engine']
    .str.extract(r'(\d+)')
    .astype(float)
)


df['Power'] = (
    df['Power']
    .str.extract(r'([\d\.]+)')
    .astype(float)
)


unit_multipliers = {
    'lakh':      1,
    'lac':       1,
    'cr':      100,
    'crore':   100,
}


price_parts = df['New_Price'].str.extract(r'([\d,\.]+)\s*([A-Za-z]+)', expand=True)
price_vals  = price_parts[0].str.replace(',', '').astype(float)
price_units = price_parts[1].str.lower().map(unit_multipliers).fillna(1)


df['New_Price'] = price_vals * price_units




df.to_csv("task_b_units_removed.csv", index=False)


print(df[['Mileage', 'Engine', 'Power', 'New_Price']].info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5847 entries, 0 to 5846
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Mileage    5845 non-null   float64
 1   Engine     5811 non-null   float64
 2   Power      5811 non-null   float64
 3   New_Price  815 non-null    float64
dtypes: float64(4)
memory usage: 182.8 KB
None


In [ ]:
missing_count = df.isna().sum()


missing_pct = (missing_count / len(df) * 100).round(2)


missing_summary = pd.DataFrame({
    'missing_count': missing_count,
    'missing_pct':   missing_pct
})


missing_summary = (
    missing_summary[missing_summary['missing_count'] > 0]
    .sort_values(by='missing_count', ascending=False)
)


print("Missing Data Summary:\n", missing_summary)

Missing Data Summary:
            missing_count  missing_pct
New_Price           5032        86.06
Seats                 38         0.65
Power                 36         0.62
Engine                36         0.62
Mileage                2         0.03


Justification :To prepare the data for analysis, we first scanned all columns for missing values. For numerical fields, we used the median to fill in the gaps because it's a reliable measure that isn’t easily affected by outliers, making it a safe choice for uneven data. For categorical fields, we filled missing entries using the mode, as it helps maintain consistency by using the most commonly occurring category.

This strategy ensured that no valuable data was discarded. Instead of removing rows or columns, we retained the original structure while filling in missing values with meaningful substitutes. After this step, the dataset was complete, balanced, and ready for further processing.

# **Question B**

In [ ]:

df.drop(columns='New_Price', inplace=True)


impute_values = {
    'Seats':   df['Seats'].mode()[0],
    'Power':   df['Power'].median(),
    'Engine':  df['Engine'].median(),
    'Mileage': df['Mileage'].median()
}


df.fillna(value=impute_values, inplace=True)


df.to_csv("task_a_missing_values.csv", index=False)

In [ ]:



total_rows = len(df)
missing_count = df.isna().sum()
missing_pct   = (missing_count / total_rows * 100).round(2)


missing_after = pd.DataFrame({
    'missing_count': missing_count,
    'missing_pct':   missing_pct
})


missing_after = missing_after[missing_after['missing_count'] > 0]


if missing_after.empty:
    print(" No missing values remain.")
else:
    print("Missing values still present after imputation:")
    print(missing_after)

 No missing values remain.


# **Question C**

In [ ]:

categorical_cols = df.select_dtypes(include='object').columns.tolist()
print("Categorical columns detected:", categorical_cols)


for col in categorical_cols:

    uniques = df[col].dropna().unique()
    count = len(uniques)
    sample = uniques[:10].tolist()
    tail = " …" if count > 10 else ""
    print(f"\n• {col} — {count} unique values. Sample: {sample}{tail}")

Categorical columns detected: ['Name', 'Location', 'Fuel_Type', 'Transmission', 'Owner_Type']

• Name — 1804 unique values. Sample: ['Hyundai Creta 1.6 CRDi SX Option', 'Honda Jazz V', 'Maruti Ertiga VDI', 'Audi A4 New 2.0 TDI Multitronic', 'Nissan Micra Diesel XV', 'Toyota Innova Crysta 2.8 GX AT 8S', 'Volkswagen Vento Diesel Comfortline', 'Tata Indica Vista Quadrajet LS', 'Maruti Ciaz Zeta', 'Honda City 1.5 V AT Sunroof'] …

• Location — 11 unique values. Sample: ['Pune', 'Chennai', 'Coimbatore', 'Jaipur', 'Mumbai', 'Kochi', 'Kolkata', 'Delhi', 'Bangalore', 'Hyderabad'] …

• Fuel_Type — 3 unique values. Sample: ['Diesel', 'Petrol', 'Electric']

• Transmission — 2 unique values. Sample: ['Manual', 'Automatic']

• Owner_Type — 4 unique values. Sample: ['First', 'Second', 'Fourth & Above', 'Third']


In [ ]:

df_ohe = pd.get_dummies(
    df,
    columns=categorical_cols,
    drop_first=True,
    prefix_sep='_',
    dtype=int
)


df_ohe.to_csv("task_c_hot_encoded.csv", index=False)


print(f"Saved 'task_c_hot_encoded.csv' — new shape: {df_ohe.shape}")
print(df_ohe.head())

Saved 'task_c_hot_encoded.csv' — new shape: (5847, 1827)
   Unnamed: 0  Year  Kilometers_Driven  Mileage  Engine   Power  Seats  Price  \
0           1  2015              41000    19.67  1582.0  126.20    5.0  12.50   
1           2  2011              46000    13.00  1199.0   88.70    5.0   4.50   
2           3  2012              87000    20.77  1248.0   88.76    7.0   6.00   
3           4  2013              40670    15.20  1968.0  140.80    5.0  17.74   
4           6  2013              86999    23.08  1461.0   63.10    5.0   3.50   

   Name_Audi A3 35 TDI Attraction  Name_Audi A3 35 TDI Premium  ...  \
0                               0                            0  ...   
1                               0                            0  ...   
2                               0                            0  ...   
3                               0                            0  ...   
4                               0                            0  ...   

   Location_Kochi  Location_K

In [ ]:

df = df.assign(Age=lambda d: pd.Timestamp.now().year - d['Year'])


output_file = "task_d_car_age_column.csv"
df.to_csv(output_file, index=False)


print(f" Saved {output_file}. Columns now include: {df.columns.tolist()}")
print("\nSample of 'Year' vs. new 'Age':")
print(df[['Year', 'Age']].head())

 Saved task_d_car_age_column.csv. Columns now include: ['Unnamed: 0', 'Name', 'Location', 'Year', 'Kilometers_Driven', 'Fuel_Type', 'Transmission', 'Owner_Type', 'Mileage', 'Engine', 'Power', 'Seats', 'Price', 'Age']

Sample of 'Year' vs. new 'Age':
   Year  Age
0  2015   10
1  2011   14
2  2012   13
3  2013   12
4  2013   12


In [ ]:

cols = ['Name', 'Year', 'Fuel_Type', 'Price']


subset = df.loc[:, cols]
print("Preview of selected columns:")
print(subset.head())


output_path = "task_e_selected_columns.csv"
subset.to_csv(output_path, index=False)
print(f"Saved {subset.shape[0]} rows and {subset.shape[1]} columns to '{output_path}'")

Preview of selected columns:
                               Name  Year Fuel_Type  Price
0  Hyundai Creta 1.6 CRDi SX Option  2015    Diesel  12.50
1                      Honda Jazz V  2011    Petrol   4.50
2                 Maruti Ertiga VDI  2012    Diesel   6.00
3   Audi A4 New 2.0 TDI Multitronic  2013    Diesel  17.74
4            Nissan Micra Diesel XV  2013    Diesel   3.50
Saved 5847 rows and 4 columns to 'task_e_selected_columns.csv'


In [ ]:
age_threshold = 5
price_threshold = 5


filtered_cars = df.query("Age > @age_threshold and Price > @price_threshold")


output_file = "task_e_filtered_cars.csv"
filtered_cars.to_csv(output_file, index=False)


print(f" {len(filtered_cars)} cars found (Age > {age_threshold} & Price > {price_threshold}L).")
print(filtered_cars.head())

 3338 cars found (Age > 5 & Price > 5L).
   Unnamed: 0                                 Name    Location  Year  \
0           1     Hyundai Creta 1.6 CRDi SX Option        Pune  2015   
2           3                    Maruti Ertiga VDI     Chennai  2012   
3           4      Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013   
5           7    Toyota Innova Crysta 2.8 GX AT 8S      Mumbai  2016   
6           8  Volkswagen Vento Diesel Comfortline        Pune  2013   

   Kilometers_Driven Fuel_Type Transmission Owner_Type  Mileage  Engine  \
0              41000    Diesel       Manual      First    19.67  1582.0   
2              87000    Diesel       Manual      First    20.77  1248.0   
3              40670    Diesel    Automatic     Second    15.20  1968.0   
5              36000    Diesel    Automatic      First    11.36  2755.0   
6              64430    Diesel       Manual      First    20.54  1598.0   

    Power  Seats  Price  Age  
0  126.20    5.0  12.50   10  
2   88.76    

# **Question D**

In [ ]:

rename_map = {
    'Price': 'Car_Price',
    'Year':  'Manufacture_Year'
}


df.rename(columns=rename_map, inplace=True)


print("Columns now are:", df.columns.tolist())


print(df.head())

Columns now are: ['Unnamed: 0', 'Name', 'Location', 'Manufacture_Year', 'Kilometers_Driven', 'Fuel_Type', 'Transmission', 'Owner_Type', 'Mileage', 'Engine', 'Power', 'Seats', 'Car_Price', 'Age']
   Unnamed: 0                              Name    Location  Manufacture_Year  \
0           1  Hyundai Creta 1.6 CRDi SX Option        Pune              2015   
1           2                      Honda Jazz V     Chennai              2011   
2           3                 Maruti Ertiga VDI     Chennai              2012   
3           4   Audi A4 New 2.0 TDI Multitronic  Coimbatore              2013   
4           6            Nissan Micra Diesel XV      Jaipur              2013   

   Kilometers_Driven Fuel_Type Transmission Owner_Type  Mileage  Engine  \
0              41000    Diesel       Manual      First    19.67  1582.0   
1              46000    Petrol       Manual      First    13.00  1199.0   
2              87000    Diesel       Manual      First    20.77  1248.0   
3              406

# **Question E**

In [ ]:

df['Price_per_Km'] = df['Car_Price'] / df['Kilometers_Driven']
print(df[['Name', 'Car_Price', 'Kilometers_Driven', 'Price_per_Km']].head())
df[['Name', 'Car_Price', 'Kilometers_Driven', 'Price_per_Km']].to_csv("task_e_mutate_price_per_km.csv", index=False)

                               Name  Car_Price  Kilometers_Driven  \
0  Hyundai Creta 1.6 CRDi SX Option      12.50              41000   
1                      Honda Jazz V       4.50              46000   
2                 Maruti Ertiga VDI       6.00              87000   
3   Audi A4 New 2.0 TDI Multitronic      17.74              40670   
4            Nissan Micra Diesel XV       3.50              86999   

   Price_per_Km  
0      0.000305  
1      0.000098  
2      0.000069  
3      0.000436  
4      0.000040  


In [ ]:

sorted_by_price = (
    df
    .sort_values('Car_Price', ascending=False)
    .loc[:, ['Name', 'Car_Price']]
)


print("Top 5 cars by price:")
print(sorted_by_price.head())


output_path = "task_e_sorted_by_price.csv"
sorted_by_price.to_csv(output_path, index=False)
print(f" Saved sorted list ({sorted_by_price.shape[0]} rows) to '{output_path}'")

Top 5 cars by price:
                                             Name  Car_Price
3952  Land Rover Range Rover 3.0 Diesel LWB Vogue     160.00
5620                   Lamborghini Gallardo Coupe     120.00
5752                       Jaguar F Type 5.0 V8 S     100.00
1457              Land Rover Range Rover Sport SE      97.07
1917                           BMW 7 Series 740Li      93.67
 Saved sorted list (5847 rows) to 'task_e_sorted_by_price.csv'


In [ ]:

fuel_summary = (
    df
    .groupby('Fuel_Type', as_index=False)['Car_Price']
    .mean()
    .rename(columns={'Car_Price': 'Avg_Car_Price'})
)


print("Average Car Price by Fuel Type:")
print(fuel_summary)


output_file = "task_e_groupby_fuel_summary.csv"
fuel_summary.to_csv(output_file, index=False)
print(f" Saved summary ({fuel_summary.shape[0]} rows) to '{output_file}'")

Average Car Price by Fuel Type:
  Fuel_Type  Avg_Car_Price
0    Diesel      12.960686
1  Electric      12.875000
2    Petrol       5.756688
 Saved summary (3 rows) to 'task_e_groupby_fuel_summary.csv'


In [ ]:
fuel_trans_summary = (
    df
    .groupby(['Fuel_Type', 'Transmission'], as_index=False)
    .agg(Avg_Car_Price=('Car_Price', 'mean'))
)


print("Average Car Price by Fuel Type and Transmission:")
print(fuel_trans_summary)


output_file = "task_e_groupby_fuel_trans_summary.csv"
fuel_trans_summary.to_csv(output_file, index=False)
print(f" Saved {fuel_trans_summary.shape[0]} rows to '{output_file}'")

Average Car Price by Fuel Type and Transmission:
  Fuel_Type Transmission  Avg_Car_Price
0    Diesel    Automatic      24.618490
1    Diesel       Manual       6.686462
2  Electric    Automatic      12.875000
3    Petrol    Automatic      11.252450
4    Petrol       Manual       4.160803
 Saved 5 rows to 'task_e_groupby_fuel_trans_summary.csv'
